### Make Figure 9 and Table 2
Select from logistic regression, decision tree, and random forest for outcome model for diabetes experiment

In [ ]:
import pickle
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score, brier_score_loss, log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.inspection import PartialDependenceDisplay

from helpers import *

In [ ]:
path_to_data = # TODO where diabetes data is located
with open(path_to_data) as f:
    df = pickle.load(f)

In [ ]:
# Filter out providers with fewer than 4 samples. Similar to load_diabetes_data function in real_data_loader.py
prv_counts = df['prv'].value_counts().sort_index() # Sorting to make deterministic
prv_counter = 0
prv_dict = dict()
orig_prvs = []
for prv in prv_counts.keys():
    if prv_counts[prv] >= 4:
        prv_dict[prv] = prv_counter
        prv_counter += 1
        orig_prvs.append(prv)
df = df[df['prv'].isin(set(list(prv_dict.keys())))]

In [ ]:
# Scale and split data
X = df[['egfr','creatinine','heart_disease','treatment_date_sec']].values
d_orig = df['prv'].values
d = np.empty(d_orig.shape)
for i in range(len(d_orig)):
    d[i] = prv_dict[d_orig[i]]
d = d.astype(int)
t = df['y'].values
train_fold_idxs, valid_fold_idxs, test_idxs = split_real_data_into_folds(d)
train_valid_idxs = np.concatenate((train_fold_idxs[0], valid_fold_idxs[0]))
train_valid_X = X[train_valid_idxs]
scaler = StandardScaler()
scaler.fit(train_valid_X)
X = scaler.transform(X)

### Table 2: AUCs on fold 3

In [ ]:
for fold_idx in range(4):
    print('Fold ' + str(fold_idx))
    X_train = X[train_fold_idxs[fold_idx]]
    t_train = t[train_fold_idxs[fold_idx]]
    X_valid = X[valid_fold_idxs[fold_idx]]
    t_valid = t[valid_fold_idxs[fold_idx]]
    X_test = X[test_idxs]
    t_test = t[test_idxs]
    
    logreg_model = train_logreg(X_train, t_train, X_valid, t_valid)
    logreg_train_pred = logreg_model.predict_proba(X_train)
    logreg_train_auc = roc_auc_score(t_train, logreg_train_pred[:,1])
    logreg_valid_pred = logreg_model.predict_proba(X_valid)
    logreg_valid_auc = roc_auc_score(t_valid, logreg_valid_pred[:,1])
    logreg_test_pred = logreg_model.predict_proba(X_test)
    logreg_test_auc = roc_auc_score(t_test, logreg_test_pred[:,1])
    
    if X_train.shape[0] > 10000:
        min_samples_leaf_options = [100, 500, 1000, 5000, 10000]
    else:
        min_samples_leaf_options = [10, 25, 100]
    dectree_model = train_decision_tree(X_train, t_train, X_valid, t_valid, min_samples_leaf_options)
    dectree_train_pred = dectree_model.predict_proba(X_train)
    dectree_train_auc = roc_auc_score(t_train, dectree_train_pred[:,1])
    dectree_valid_pred = dectree_model.predict_proba(X_valid)
    dectree_valid_auc = roc_auc_score(t_valid, dectree_valid_pred[:,1])
    dectree_test_pred = dectree_model.predict_proba(X_test)
    dectree_test_auc = roc_auc_score(t_test, dectree_test_pred[:,1])
    
    randforest_model = train_random_forest(X_train, t_train, X_valid, t_valid)
    randforest_train_pred = randforest_model.predict_proba(X_train)
    randforest_train_auc = roc_auc_score(t_train, randforest_train_pred[:,1])
    randforest_valid_pred = randforest_model.predict_proba(X_valid)
    randforest_valid_auc = roc_auc_score(t_valid, randforest_valid_pred[:,1])
    randforest_test_pred = randforest_model.predict_proba(X_test)
    randforest_test_auc = roc_auc_score(t_test, randforest_test_pred[:,1])
    
    print('Logistic regression AUC: Train {0:.4f}'.format(logreg_train_auc) 
          + ', Valid {0:.4f}'.format(logreg_valid_auc) + ', Test {0:.4f}'.format(logreg_test_auc))
    print('Decision tree AUC: Train {0:.4f}'.format(dectree_train_auc) + ', Valid {0:.4f}'.format(dectree_valid_auc)
          + ', Test {0:.4f}'.format(dectree_test_auc))
    print('Random forest AUC: Train {0:.4f}'.format(randforest_train_auc) 
          + ', Valid {0:.4f}'.format(randforest_valid_auc) + ', Test {0:.4f}'.format(randforest_test_auc))

### Figure 9: partial dependence plot of selected random forest model with features at original scale

In [ ]:
unscaled_X = df[['egfr','creatinine','heart_disease','treatment_date_sec']].values
unscaled_X_train = unscaled_X[train_fold_idxs[3]]
t_train = t[train_fold_idxs[3]]
unscaled_X_valid = unscaled_X[valid_fold_idxs[3]]
t_valid = t[valid_fold_idxs[3]]

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, sharey=True, figsize=(6,6))
feature_names=['eGFR', 'Creatinine', 'Heart disease', 'Treatment date']
plt.rcParams.update({'font.size': 10})
PartialDependenceDisplay.from_estimator(randforest_model, unscaled_X_train, [0,1,2,3], feature_names=feature_names, 
                                        ax=ax)
plt.tight_layout()
plt.savefig('random_forest_pdp_correct_scale.pdf')
plt.show()